In [0]:
#importing the necessary libraries

import pandas as pd
import numpy as np
import scipy #used for mathematical functions


In [29]:
#import spacy and nltk libraries
import spacy
from spacy import displacy #display the entities
import nltk

#to use Python’s raw string notation for regular expression patterns
import re 

import nltk
nltk.download()



NLTK Downloader
---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> d

Download which package (l=list; x=cancel)?
  Identifier> popular
       | 
       | Downloading package cmudict to /root/nltk_data...
       |   Package cmudict is already up-to-date!
       | Downloading package gazetteers to /root/nltk_data...
       |   Package gazetteers is already up-to-date!
       | Downloading package genesis to /root/nltk_data...
       |   Package genesis is already up-to-date!
       | Downloading package gutenberg to /root/nltk_data...
       |   Package gutenberg is already up-to-date!
       | Downloading package inaugural to /root/nltk_data...
       |   Package inaugural is already up-to-date!
       | Downloading package movie_reviews to /root/nltk_data...
       |   Package movie_reviews is already up-

True

In [0]:
import pandas as pd
import numpy as np
import scipy

import spacy
from spacy import displacy
import nltk
import re

from nltk.corpus import stopwords
stopwords = stopwords.words('english')

from collections import Counter
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.feature_extraction.text import CountVectorizer
from sklearn import ensemble
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

np.random.seed(27)

In [0]:
%matplotlib inline

plt.rcParams['figure.figsize'] = [20.0, 7.0]
plt.rcParams.update({'font.size': 22})

sns.set_palette('bright')
sns.set_style('white')
sns.set_context('talk', font_scale=0.8)


In [32]:
# read in the data
raw_data = pd.read_csv('/content/drive/My Drive/mtsamples_v2.csv')
raw_data.head()

,Unnamed: 0,description,medical_specialty,sample_name,transcription,keywords
0,0,A 23-year-old white female presents with comp...,Allergy / Immunology,Allergic Rhinitis,"SUBJECTIVE:, This 23-year-old white female pr...","allergy / immunology, allergic rhinitis, aller..."
1,1,Consult for laparoscopic gastric bypass.,Bariatrics,Laparoscopic Gastric Bypass Consult - 2,"PAST MEDICAL HISTORY:, He has difficulty climb...","bariatrics, laparoscopic gastric bypass, weigh..."
2,2,Consult for laparoscopic gastric bypass.,Bariatrics,Laparoscopic Gastric Bypass Consult - 1,"HISTORY OF PRESENT ILLNESS: , I have seen ABC ...","bariatrics, laparoscopic gastric bypass, heart..."
3,3,2-D M-Mode. Doppler.,Cardiovascular / Pulmonary,2-D Echocardiogram - 1,"2-D M-MODE: , ,1. Left atrial enlargement wit...","cardiovascular / pulmonary, 2-d m-mode, dopple..."
4,4,2-D Echocardiogram,Cardiovascular / Pulmonary,2-D Echocardiogram - 2,1. The left ventricular cavity size and wall ...,"cardiovascular / pulmonary, 2-d, doppler, echo..."


In [33]:

#Clean, Normalize, and Pre-process Text
#Before attempting to apply any machine learning algorithms we must first pre-process our text. This step includes removing stop-words and unnecessary punctuations.

#Pre-processing techniques to try:

#tokenization
#tagging
#chunking
#lemmatization
# visualizing a sample text
sample_text = raw_data.iloc[0,4]
sample_text

'SUBJECTIVE:,  This 23-year-old white female presents with complaint of allergies.  She used to have allergies when she lived in Seattle but she thinks they are worse here.  In the past, she has tried Claritin, and Zyrtec.  Both worked for short time but then seemed to lose effectiveness.  She has used Allegra also.  She used that last summer and she began using it again two weeks ago.  It does not appear to be working very well.  She has used over-the-counter sprays but no prescription nasal sprays.  She does have asthma but doest not require daily medication for this and does not think it is flaring up.,MEDICATIONS: , Her only medication currently is Ortho Tri-Cyclen and the Allegra.,ALLERGIES: , She has no known medicine allergies.,OBJECTIVE:,Vitals:  Weight was 130 pounds and blood pressure 124/78.,HEENT:  Her throat was mildly erythematous without exudate.  Nasal mucosa was erythematous and swollen.  Only clear drainage was seen.  TMs were clear.,Neck:  Supple without adenopathy.,

In [34]:
# trying out potential tokenizers
# visualizing output with nltk default sentence tokenizer
nltk.sent_tokenize(text=sample_text)

['SUBJECTIVE:,  This 23-year-old white female presents with complaint of allergies.',
 'She used to have allergies when she lived in Seattle but she thinks they are worse here.',
 'In the past, she has tried Claritin, and Zyrtec.',
 'Both worked for short time but then seemed to lose effectiveness.',
 'She has used Allegra also.',
 'She used that last summer and she began using it again two weeks ago.',
 'It does not appear to be working very well.',
 'She has used over-the-counter sprays but no prescription nasal sprays.',
 'She does have asthma but doest not require daily medication for this and does not think it is flaring up.,MEDICATIONS: , Her only medication currently is Ortho Tri-Cyclen and the Allegra.,ALLERGIES: , She has no known medicine allergies.,OBJECTIVE:,Vitals:  Weight was 130 pounds and blood pressure 124/78.,HEENT:  Her throat was mildly erythematous without exudate.',
 'Nasal mucosa was erythematous and swollen.',
 'Only clear drainage was seen.',
 'TMs were clear.,

In [35]:
# viewing output for RegExpTokenizer
SENTENCE_TOKENS_PATTERN = r'(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<![A-Z]\.)(?<=\.|\?|\!)\s'
regex_st = nltk.tokenize.RegexpTokenizer(
            pattern=SENTENCE_TOKENS_PATTERN,
            gaps=True)
regex_st.tokenize(sample_text)

['SUBJECTIVE:,  This 23-year-old white female presents with complaint of allergies.',
 ' She used to have allergies when she lived in Seattle but she thinks they are worse here.',
 ' In the past, she has tried Claritin, and Zyrtec.',
 ' Both worked for short time but then seemed to lose effectiveness.',
 ' She has used Allegra also.',
 ' She used that last summer and she began using it again two weeks ago.',
 ' It does not appear to be working very well.',
 ' She has used over-the-counter sprays but no prescription nasal sprays.',
 ' She does have asthma but doest not require daily medication for this and does not think it is flaring up.,MEDICATIONS: , Her only medication currently is Ortho Tri-Cyclen and the Allegra.,ALLERGIES: , She has no known medicine allergies.,OBJECTIVE:,Vitals:  Weight was 130 pounds and blood pressure 124/78.,HEENT:  Her throat was mildly erythematous without exudate.',
 ' Nasal mucosa was erythematous and swollen.',
 ' Only clear drainage was seen.',
 ' TMs w

In [36]:
# sentence tokenizer was not able to correctly tokenize the sentences. So applying some cleaning functions to see if we can get a better result.
def tokenize_text(text):
    sentences = nltk.sent_tokenize(text)
    word_tokens = [nltk.word_tokenize(sentence) for sentence in sentences] 
    return word_tokens

def remove_characters_before_tokenization(sentence, keep_dash=True):
    sentence = sentence.strip()
    if keep_dash:
        # removing most special characters
        # we want to keep at least - and / as these are important in this text
        PATTERN = r'[?|$|&|*|%|@|(|)|~|,|]'
        filtered_sentence = re.sub(PATTERN, r'', sentence)
        # replacing : with whitespace to ensure words don't run together
        filtered_sentence = re.sub(r'[:]', ' ', filtered_sentence)
        # adding whitespace after . to help tokenizer capture correct sentences
        filtered_sentence = re.sub(r'[.]', ' ', filtered_sentence)
    else:
        PATTERN = r'[^a-zA-Z0-9 ]'
        filtered_sentence = re.sub(PATTERN, r'', sentence)
    return filtered_sentence

# visualizing test output
cleaned_sample = remove_characters_before_tokenization(sample_text) 
cleaned_sample

'SUBJECTIVE   This 23-year-old white female presents with complaint of allergies   She used to have allergies when she lived in Seattle but she thinks they are worse here   In the past she has tried Claritin and Zyrtec   Both worked for short time but then seemed to lose effectiveness   She has used Allegra also   She used that last summer and she began using it again two weeks ago   It does not appear to be working very well   She has used over-the-counter sprays but no prescription nasal sprays   She does have asthma but doest not require daily medication for this and does not think it is flaring up MEDICATIONS   Her only medication currently is Ortho Tri-Cyclen and the Allegra ALLERGIES   She has no known medicine allergies OBJECTIVE Vitals   Weight was 130 pounds and blood pressure 124/78 HEENT   Her throat was mildly erythematous without exudate   Nasal mucosa was erythematous and swollen   Only clear drainage was seen   TMs were clear Neck   Supple without adenopathy Lungs   Clea

In [37]:
nltk.sent_tokenize(text=cleaned_sample)

['SUBJECTIVE   This 23-year-old white female presents with complaint of allergies   She used to have allergies when she lived in Seattle but she thinks they are worse here   In the past she has tried Claritin and Zyrtec   Both worked for short time but then seemed to lose effectiveness   She has used Allegra also   She used that last summer and she began using it again two weeks ago   It does not appear to be working very well   She has used over-the-counter sprays but no prescription nasal sprays   She does have asthma but doest not require daily medication for this and does not think it is flaring up MEDICATIONS   Her only medication currently is Ortho Tri-Cyclen and the Allegra ALLERGIES   She has no known medicine allergies OBJECTIVE Vitals   Weight was 130 pounds and blood pressure 124/78 HEENT   Her throat was mildly erythematous without exudate   Nasal mucosa was erythematous and swollen   Only clear drainage was seen   TMs were clear Neck   Supple without adenopathy Lungs   Cle

In [38]:
#With a little cleaning, the sentence tokenizer performed much better! Let's now modify our cleaning function from above. We should add functionality to remove stop words and further normalize the text by removing capitol letters.

#From viewing the sample text, I think the categories that are written in all caps may be an important feature, but lets go ahead and lowercase everything for now.

def normalize_text(doc):
    doc = doc.lower()
    # removing most special characters
    # we want to keep at least - and / as these are important in this text
    PATTERN = r'[?|$|&|*|%|@|(|)|~|,|]'
    doc = re.sub(PATTERN, r'', doc)
    # replacing : with whitespace to ensure words don't run together
    doc = re.sub(r'[:]', ' ', doc)
    # replacing . with whitespace to help tokenizer capture correct sentences
    doc = re.sub(r'[.]', ' ', doc)
    # removing extra whitespace
    doc = doc.replace('   ', ' ')
    doc = doc.strip()
    
    return doc

sample_normalized = normalize_text(sample_text)
sample_normalized

'subjective this 23-year-old white female presents with complaint of allergies she used to have allergies when she lived in seattle but she thinks they are worse here in the past she has tried claritin and zyrtec both worked for short time but then seemed to lose effectiveness she has used allegra also she used that last summer and she began using it again two weeks ago it does not appear to be working very well she has used over-the-counter sprays but no prescription nasal sprays she does have asthma but doest not require daily medication for this and does not think it is flaring up medications her only medication currently is ortho tri-cyclen and the allegra allergies she has no known medicine allergies objective vitals weight was 130 pounds and blood pressure 124/78 heent her throat was mildly erythematous without exudate nasal mucosa was erythematous and swollen only clear drainage was seen tms were clear neck supple without adenopathy lungs clear assessment allergic rhinitis plan 

In [43]:
#Download Available pretrained statistical models for English
!python -m spacy download en_core_web_lg
!python -m spacy download en

     |████████████████████████████████| 827.9MB 1.2MB/s 
  Created wheel for en-core-web-lg: filename=en_core_web_lg-2.2.5-cp36-none-any.whl size=829180944 sha256=c346afd8a4768df3651d0f5428b416408cb55e1082440f76c9f97e5f5a197321
  Stored in directory: /tmp/pip-ephem-wheel-cache-kztvfrx1/wheels/2a/c1/a6/fc7a877b1efca9bc6a089d6f506f16d3868408f9ff89f8dbfc
Successfully built en-core-web-lg
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_lg')
     |████████████████████████████████| 12.0MB 779kB/s 
  Created wheel for en-core-web-sm: filename=en_core_web_sm-2.2.5-cp36-none-any.whl size=12011742 sha256=d37220166faf435930b08939c239dd4a0510e6e0b1ae4b6ce792c5ca7671f780
  Stored in directory: /tmp/pip-ephem-wheel-cache-yto__4et/wheels/6a/47/fb/6b5a0b8906d8e8779246c67d4658fd8a544d4a03a75520197a
Successfully built en-core-web-sm
  Found existing installation: en-core-web-sm 2.1.0
    Uninstalling en-core-web-sm-2.1.0:
      Successfully uninstalled en-co

In [0]:
# tokenize the transcriptions with spacy

nlp = spacy.load('en')
#nlp = spacy.load('en_core_web_lg')

In [0]:
def lemmatize_text(text):
    text = nlp(text)
    text = ' '.join([word.lemma_ if word.lemma_ != '-PRON-' else word.text for word in text])
    return text

In [50]:
# lemmatizing corpus

lemma_sample = lemmatize_text(sample_normalized)
print(type(lemma_sample))
lemma_sample = nlp(lemma_sample)
print(type(lemma_sample))
lemma_sample

<class 'str'>
<class 'spacy.tokens.doc.Doc'>


subjective this 23-year - old white female present with complaint of allergy she use to have allergy when she live in seattle but she think they be bad here in the past she have try claritin and zyrtec both work for short time but then seem to lose effectiveness she have use allegra also she use that last summer and she begin use it again two week ago it do not appear to be work very well she have use over - the - counter spray but no prescription nasal spray she do have asthma but doest not require daily medication for this and do not think it be flare up medication her only medication currently be ortho tri - cyclen and the allegra allergy she have no know medicine allergy objective vital weight be 130 pound and blood pressure 124/78 heent her throat be mildly erythematous without exudate nasal mucosa be erythematous and swollen only clear drainage be see tms be clear neck supple without adenopathy lung clear assessment allergic rhinitis plan 1 she will try zyrtec instead of allegra 

In [51]:
tokens = [token.text for token in lemma_sample if not token.is_stop]
tokens[0:10]

['subjective',
 '23-year',
 '-',
 'old',
 'white',
 'female',
 'present',
 'complaint',
 'allergy',
 'use']

In [52]:

# using spacy to visualize a named entity annotated transcription
displacy.render(lemma_sample, style='ent', jupyter=True)

In [53]:

# visualizing the dependency tree for the first sentenece in our sample transcription

doc = nlp('subjective this 23-year - old white female present with complaint of allergy')
displacy.render(doc, style='dep', jupyter=True, options={'distance': 90})

In [54]:

# first checking our transcriptions for null values
raw_data.transcription.isnull().sum()

33

In [55]:

# dropping the 33 rows with no transcription text
df = raw_data.copy()
df.dropna(subset=['transcription'], inplace=True)

print('Transcriptions with null values:')
print(df.transcription.isnull().sum())

df.head()

Transcriptions with null values:
0


,Unnamed: 0,description,medical_specialty,sample_name,transcription,keywords
0,0,A 23-year-old white female presents with comp...,Allergy / Immunology,Allergic Rhinitis,"SUBJECTIVE:, This 23-year-old white female pr...","allergy / immunology, allergic rhinitis, aller..."
1,1,Consult for laparoscopic gastric bypass.,Bariatrics,Laparoscopic Gastric Bypass Consult - 2,"PAST MEDICAL HISTORY:, He has difficulty climb...","bariatrics, laparoscopic gastric bypass, weigh..."
2,2,Consult for laparoscopic gastric bypass.,Bariatrics,Laparoscopic Gastric Bypass Consult - 1,"HISTORY OF PRESENT ILLNESS: , I have seen ABC ...","bariatrics, laparoscopic gastric bypass, heart..."
3,3,2-D M-Mode. Doppler.,Cardiovascular / Pulmonary,2-D Echocardiogram - 1,"2-D M-MODE: , ,1. Left atrial enlargement wit...","cardiovascular / pulmonary, 2-d m-mode, dopple..."
4,4,2-D Echocardiogram,Cardiovascular / Pulmonary,2-D Echocardiogram - 2,1. The left ventricular cavity size and wall ...,"cardiovascular / pulmonary, 2-d, doppler, echo..."


In [0]:
# also dropping the unnnamed column as it doesnt provide any value
df.drop(columns=['Unnamed: 0'], axis=1, inplace=True)

In [0]:
normalize_corpus = np.vectorize(normalize_text)
df['transcriptions_normal'] = normalize_corpus(df.transcription)

In [0]:
df['lemma_transcription'] = [lemmatize_text(item) for item in df.transcriptions_normal]

In [59]:
df.head()

,description,medical_specialty,sample_name,transcription,keywords,transcriptions_normal,lemma_transcription
0,A 23-year-old white female presents with comp...,Allergy / Immunology,Allergic Rhinitis,"SUBJECTIVE:, This 23-year-old white female pr...","allergy / immunology, allergic rhinitis, aller...",subjective this 23-year-old white female prese...,subjective this 23-year - old white female pre...
1,Consult for laparoscopic gastric bypass.,Bariatrics,Laparoscopic Gastric Bypass Consult - 2,"PAST MEDICAL HISTORY:, He has difficulty climb...","bariatrics, laparoscopic gastric bypass, weigh...",past medical history he has difficulty climbi...,past medical history he have difficulty clim...
2,Consult for laparoscopic gastric bypass.,Bariatrics,Laparoscopic Gastric Bypass Consult - 1,"HISTORY OF PRESENT ILLNESS: , I have seen ABC ...","bariatrics, laparoscopic gastric bypass, heart...",history of present illness i have seen abc tod...,history of present illness i have see abc toda...
3,2-D M-Mode. Doppler.,Cardiovascular / Pulmonary,2-D Echocardiogram - 1,"2-D M-MODE: , ,1. Left atrial enlargement wit...","cardiovascular / pulmonary, 2-d m-mode, dopple...",2-d m-mode 1 left atrial enlargement with left...,2-d m - mode 1 leave atrial enlargement with l...
4,2-D Echocardiogram,Cardiovascular / Pulmonary,2-D Echocardiogram - 2,1. The left ventricular cavity size and wall ...,"cardiovascular / pulmonary, 2-d, doppler, echo...",1 the left ventricular cavity size and wall th...,1 the left ventricular cavity size and wall th...


In [60]:
# loading nltk stopwords
stop_words = nltk.corpus.stopwords.words('english')

# from visual inspection of corpus adding to our stopword list
more_stops = ['aa', 'ab', 'abc', 'abcd', 'xxx', 'xyz', 'xii', 'dr', 'x', 'mg', 'p', 'ml']

stop_words = stop_words + more_stops
print (stop_words)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [61]:
from nltk.tokenize.toktok import ToktokTokenizer
tokenizer = ToktokTokenizer()

# remove numbers and stopwords
def remove_stopwords_and_nums(text):
    tokens = tokenizer.tokenize(text)
    # remove all non-alpha characters
    tokens = filter(lambda x: x.isalpha(), tokens)
    tokens = [token.strip() for token in tokens]
    # filter out stopwords
    filtered_tokens = [token for token in tokens if token not in stop_words]
    filtered_text = ' '.join(filtered_tokens)
    return filtered_text

remove_stopwords_and_nums(lemma_sample)

'subjective old white female present complaint allergy use allergy live seattle think bad past try claritin zyrtec work short time seem lose effectiveness use allegra also use last summer begin use two week ago appear work well use counter spray prescription nasal spray asthma doest require daily medication think flare medication medication currently ortho tri cyclen allegra allergy know medicine allergy objective vital weight pound blood pressure heent throat mildly erythematous without exudate nasal mucosa erythematous swollen clear drainage see tms clear neck supple without adenopathy lung clear assessment allergic rhinitis plan try zyrtec instead allegra another option use loratadine think prescription coverage may cheap sample nasonex two spray nostril give three week prescription write well'

In [62]:
df['alpha_only'] = [remove_stopwords_and_nums(item) for item in df.lemma_transcription]
df['alpha_tokens'] = [tokenizer.tokenize(text) for text in df.alpha_only]
df.sample(5)

,description,medical_specialty,sample_name,transcription,keywords,transcriptions_normal,lemma_transcription,alpha_only,alpha_tokens
1820,"Plantar fascitis, left foot. Partial plantar...",Podiatry,Plantar Fasciotomy,"PREOPERATIVE DIAGNOSIS:, Plantar fascitis, le...","podiatry, foot, plantar fasciotomy, plantar fa...",preoperative diagnosis plantar fascitis left f...,preoperative diagnosis plantar fascitis leave ...,preoperative diagnosis plantar fascitis leave ...,"[preoperative, diagnosis, plantar, fascitis, l..."
3994,"Incision and drainage, first metatarsal head,...",Discharge Summary,Abscess with Cellulitis - Discharge Summary,"ADMITTING DIAGNOSIS:, Abscess with cellulitis...","discharge summary, accutane, metatarsal head l...",admitting diagnosis abscess with cellulitis le...,admit diagnosis abscess with cellulitis leave ...,admit diagnosis abscess cellulitis leave foot ...,"[admit, diagnosis, abscess, cellulitis, leave,..."
3507,Appendicitis. Laparoscopic appendectomy. C...,Gastroenterology,Laparoscopic Appendectomy - 3,"PREOPERATIVE DIAGNOSIS: , Appendicitis.,POSTOP...","gastroenterology, foley catheter, co2 insuffla...",preoperative diagnosis appendicitis postoperat...,preoperative diagnosis appendicitis postoperat...,preoperative diagnosis appendicitis postoperat...,"[preoperative, diagnosis, appendicitis, postop..."
2545,Invasive carcinoma of left breast. Left modif...,Obstetrics / Gynecology,Radical Mastectomy,"PREOPERATIVE DIAGNOSIS:, Invasive carcinoma o...","obstetrics / gynecology, invasive carcinoma, c...",preoperative diagnosis invasive carcinoma of l...,preoperative diagnosis invasive carcinoma of l...,preoperative diagnosis invasive carcinoma left...,"[preoperative, diagnosis, invasive, carcinoma,..."
1739,Carotid artery angiograms.,Radiology,Bilateral Carotid Angiography,"PROCEDURE PERFORMED:,1. Selective ascending a...","radiology, aortic arch angiogram, carotid arte...",procedure performed 1 selective ascending aort...,procedure perform 1 selective ascend aortic ar...,procedure perform selective ascend aortic arch...,"[procedure, perform, selective, ascend, aortic..."


In [63]:
# creating a list of the total vocabulary for the entire transcriptions corpus
# extend creates a flat list of words
total_vocab = []
for token in df.alpha_tokens:
    total_vocab.extend(token)

# removing duplicate words using set (unordered collection of distinct objects)
total_vocab = list(set(total_vocab))
total_vocab = pd.DataFrame({'words':total_vocab}, index=total_vocab)
total_vocab.sample(5)

,words
epineurectomy,epineurectomy
parchment,parchment
prepyloric,prepyloric
salisbury,salisbury
dictate,dictate


In [64]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(max_features=10000)
cv_matrix = cv.fit_transform(df.alpha_only)
cv_matrix = cv_matrix.toarray()
cv_matrix[0:10]

array([[0, 0, 0, ..., 0, 2, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [65]:
# get all unique words in corpus
vocab = cv.get_feature_names()
# show feature vectors
pd.DataFrame(cv_matrix, columns=vocab).head()

,abandon,abbreviate,abcdob,abd,abdoman,abdomen,abdominal,abdominally,abdominis,abdominus,abds,abduct,abduction,abductor,abductovalgus,abg,abilify,ability,ablate,ablation,able,abnormal,abnormality,abnormally,abort,abortion,abrade,abrasion,abrupt,abscess,absence,absent,absolute,absolutely,absorb,absorbable,absorbing,abundant,abuse,abusive,...,yellow,yellowish,yes,yesterday,yet,yield,yogurt,york,young,youngswick,yr,yy,yyyy,yyyydear,zanaflex,zantac,zaontz,zaroxolyn,zegerid,zeiss,zephyr,zero,zestril,zetia,zigzag,zimmer,zinc,zithromax,zocor,zofran,zoloft,zone,zonegran,zoster,zosyn,zygoma,zygomatic,zyprexa,zyrtec,zyvox
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [66]:
bv = CountVectorizer(ngram_range=(2,3), max_features=10000)
bv_matrix = bv.fit_transform(df.alpha_only)
bv_matrix = bv_matrix.toarray()

vocab = bv.get_feature_names()
pd.DataFrame(bv_matrix, columns=vocab).head()

,abdoman benign,abdoman insufflate,abdoman pelvi,abdoman preppe,abdoman preppe drape,abdoman show,abdoman soft,abdoman soft nondistende,abdoman soft nontender,abdomen benign,abdomen nontender,abdomen obese,abdomen pelvi,abdomen preppe,abdomen preppe drape,abdomen soft,abdomen soft extremity,abdomen soft nontender,abdominal aorta,abdominal aortic,abdominal aortic aneurysm,abdominal cavity,abdominal discomfort,abdominal hysterectomy,abdominal incision,abdominal pain,abdominal pain nausea,abdominal scar,abdominal surgery,abdominal wall,able ambulate,able complete,able get,able identify,able pass,able perform,able place,able walk,abnormal finding,abnormal reveal,...,wound close,wound close layer,wound closure,wound copiously,wound copiously irrigate,wound culture,wound dress,wound edge,wound flush,wound infection,wound inspect,wound irrigate,wound irrigate antibiotic,wound irrigate bacitracin,wound irrigate close,wound irrigate normal,wound pack,wound right,wound thoroughly,wound thoroughly irrigate,wound vac,wrap around,write consent,write consent obtain,write patient,xl daily,xylocaine epinephrine,year age,year ago,year also,year history,year old,year old female,year past,year patient,year prior,year well,young age,young lady,young man
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,0,0,1,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [67]:
from sklearn.feature_extraction.text import TfidfVectorizer

tv = TfidfVectorizer(max_features=10000,
                     min_df=2, #use only words appearing at least twice
                     max_df=0.7, #drop words occuring in a majority of the documents
                     stop_words='english',
                     ngram_range=(1,3)
                    )
%time tv_matrix = tv.fit_transform(df.alpha_only)
tv_matrix = tv_matrix.toarray()

# vocabulary of features in tfidf matrix
vocab = tv.get_feature_names()
pd.DataFrame(np.round(tv_matrix, 2), columns=vocab).head()

CPU times: user 7.46 s, sys: 153 ms, total: 7.61 s
Wall time: 7.7 s


,abd,abdoman,abdoman insufflate,abdoman pelvi,abdoman preppe,abdoman preppe drape,abdoman soft,abdoman soft nontender,abdomen,abdomen obese,abdomen pelvi,abdomen soft,abdomen soft nontender,abdominal,abdominal aorta,abdominal aortic,abdominal cavity,abdominal hysterectomy,abdominal pain,abdominal pain nausea,abdominal wall,abds,abduction,abductor,abductovalgus,ability,ablation,able,able pass,able walk,abnormal,abnormality,abnormality identify,abnormality note,abortion,abrasion,abscess,absence,absent,absolutely,...,wound flush,wound irrigate,wound thoroughly,wrap,wrist,write,write consent,wt,xanax,xeroform,xiphoid,xl,xopenex,xx,xylocaine,xylocaine epinephrine,year,year age,year ago,year history,year old,year old female,year patient,year prior,yellow,yes,yesterday,young,young age,young man,yyyy,zantac,zero,zithromax,zocor,zofran,zoloft,zone,zygoma,zyrtec
0,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.08,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.21
1,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.07,0.00,0.05,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00
2,0.0,0.03,0.0,0.0,0.0,0.0,0.04,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.03,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.14,0.05,0.09,0.0,0.04,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00
3,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00
4,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.04,0.04,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00


In [68]:
from sklearn.metrics.pairwise import cosine_similarity

similarity_matrix = 1 - cosine_similarity(tv_matrix)
similarity_df = pd.DataFrame(similarity_matrix)
similarity_df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,4926,4927,4928,4929,4930,4931,4932,4933,4934,4935,4936,4937,4938,4939,4940,4941,4942,4943,4944,4945,4946,4947,4948,4949,4950,4951,4952,4953,4954,4955,4956,4957,4958,4959,4960,4961,4962,4963,4964,4965
0,-2.220446e-16,9.268870e-01,9.184633e-01,0.998089,0.983132,0.985114,0.975025,0.997800,0.980865,1.000000,0.982716,0.996752,0.977558,0.925423,0.988047,0.989564,0.981362,0.927371,0.995125,0.945810,0.996658,0.974716,0.984592,0.984458,0.972675,1.000000,0.983482,0.998923,0.935851,0.975207,0.974754,0.931938,0.967211,0.962618,0.947349,0.997452,0.987935,0.986087,0.980917,0.934041,...,0.989663,0.938541,0.986044,0.985915,0.981545,0.975542,0.959703,1.000000,0.995819,0.975114,0.981076,0.992175,0.982652,0.991542,0.957652,0.925488,0.988104,0.979294,0.954459,0.956306,0.961791,0.928619,0.979779,0.947688,1.0,0.957269,0.929200,0.972363,0.992494,0.962547,0.936609,0.976519,0.946966,0.978687,0.879658,0.812197,0.986616,0.763531,0.932574,0.871225
1,9.268870e-01,4.440892e-16,3.799889e-01,0.982352,0.976620,0.977078,0.984560,0.960685,0.994357,1.000000,0.952210,0.984258,0.952409,0.800113,0.993302,0.942366,0.978337,0.927882,0.985545,0.928336,0.982689,0.946565,0.993534,0.994633,1.000000,0.972971,0.944690,0.980992,0.969207,0.976102,0.944647,0.900288,0.888521,0.985110,0.948034,0.989439,0.998627,0.984271,0.979738,0.891260,...,0.982911,0.918343,0.971273,0.923038,0.965984,0.980397,0.851094,0.981076,0.967074,0.936671,0.937491,0.982748,0.772884,0.968020,0.802486,0.849401,0.923900,0.845404,0.869450,0.913575,0.912530,0.883758,0.902459,0.819059,1.0,0.911462,0.782427,0.937145,0.966029,0.906725,0.845821,0.954164,0.935153,0.959558,0.819711,0.893127,0.956926,0.889905,0.894812,0.875405
2,9.184633e-01,3.799889e-01,-2.220446e-16,0.979840,0.974032,0.934907,0.970501,0.956672,0.985630,0.992026,0.891922,0.983436,0.952496,0.771524,0.984461,0.800619,0.954842,0.897660,0.977525,0.933033,0.987690,0.864154,0.996356,0.991638,1.000000,0.987716,0.933705,0.984445,0.945197,0.974793,0.938223,0.905656,0.901287,0.986381,0.953997,0.990237,0.996360,0.963910,0.971676,0.881951,...,0.984950,0.912963,0.971727,0.929968,0.958651,0.945827,0.869010,0.982620,0.978011,0.945534,0.940147,0.981483,0.783252,0.965814,0.843735,0.847582,0.909541,0.866757,0.751067,0.837934,0.734802,0.800774,0.907103,0.794041,1.0,0.834602,0.738456,0.947802,0.976954,0.923264,0.755967,0.951294,0.934751,0.959854,0.867326,0.873723,0.943093,0.880205,0.894305,0.882268
3,9.980886e-01,9.823523e-01,9.798402e-01,0.000000,0.490351,0.985569,0.996800,0.702948,1.000000,0.347005,0.998109,0.561157,0.977258,0.998187,0.991024,0.983704,1.000000,0.998255,0.997407,0.996869,0.991698,0.993877,0.996520,0.992928,1.000000,0.994662,0.984286,1.000000,0.995957,0.996193,1.000000,0.986673,0.975443,0.997651,1.000000,0.944061,0.991770,0.992885,0.990096,0.975068,...,0.954900,0.841345,0.977219,0.853295,0.982060,0.992822,0.742870,0.930485,0.964185,0.926557,0.912631,0.981374,0.976823,0.928713,0.990251,0.959341,0.935045,0.859578,0.981267,0.986877,0.996516,0.995400,0.900738,0.997508,1.0,0.996666,0.998158,0.954973,0.984766,0.977185,0.993518,0.952217,0.974251,0.964755,1.000000,0.975090,0.997216,0.986773,0.995480,0.989234
4,9.831322e-01,9.766205e-01,9.740319e-01,0.490351,0.000000,0.983965,0.991710,0.643658,0.999193,0.567953,0.996976,0.563323,0.965834,0.985262,0.991225,0.984062,0.979654,0.996274,0.995268,0.993880,0.983650,0.993461,0.996403,0.993122,1.000000,0.994482,0.978976,0.985745,0.983684,0.997971,1.000000,0.977372,0.961243,0.991429,1.000000,0.936196,0.982481,0.979053,0.941695,0.965882,...,0.923110,0.840973,0.935387,0.796605,0.960669,0.979345,0.801692,0.940878,0.954432,0.858406,0.855988,0.950807,0.969584,0.934169,0.970125,0.964208,0.951164,0.898015,0.978594,0.983747,0.995708,0.993229,0.869727,0.981305,1.0,0.997778,0.985370,0.939146,0.972352,0.960311,0.991423,0.918232,0.966673,0.937751,1.000000,0.956312,0.990630,0.970168,0.986655,0.973724
